In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

# =============================================
# Dataset Path  (UPDATE BASED ON YOUR COMPUTER)
# =============================================
DATASET_DIR = r"C:\data_imgtype"   # <--- Your dataset path!!

train_dir = DATASET_DIR + r"\train"
val_dir   = DATASET_DIR + r"\val"

# =============================================
# Data Transformations
# =============================================
IMG_SIZE = 224

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Show class indexing result:
print("Class Mapping:", train_dataset.class_to_idx)
# Example:
# {'chest_xray': 0, 'fundus': 1, 'other': 2}

# =============================================
# Load Pretrained ResNet18
# =============================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

# Replace final layer for 3 classes
model.fc = nn.Linear(model.fc.in_features, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# =============================================
# Training Loop
# =============================================
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    train_accuracy = correct / total * 100
    val_correct = 0
    val_total = 0

    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_accuracy = val_correct / val_total * 100

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {train_loss:.4f} "
          f"Train Acc: {train_accuracy:.2f}% "
          f"Val Acc: {val_accuracy:.2f}%")

# =============================================
# Save Model
# =============================================
MODEL_PATH = "imgtype_resnet18.pth"
torch.save(model.state_dict(), MODEL_PATH)
print("🎯 Training complete! Model saved as:", MODEL_PATH)


Class Mapping: {'chest_xray': 0, 'fundus': 1, 'other': 2}


C:\Users\Devraj\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Devraj\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Devraj/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [03:37<00:00, 216kB/s]


Epoch [1/10] Train Loss: 5.6071 Train Acc: 93.11% Val Acc: 100.00%
Epoch [2/10] Train Loss: 0.2955 Train Acc: 100.00% Val Acc: 99.71%
Epoch [3/10] Train Loss: 0.1388 Train Acc: 100.00% Val Acc: 99.71%
Epoch [4/10] Train Loss: 3.1276 Train Acc: 99.29% Val Acc: 100.00%
Epoch [5/10] Train Loss: 0.5422 Train Acc: 99.76% Val Acc: 100.00%
Epoch [6/10] Train Loss: 1.6362 Train Acc: 99.29% Val Acc: 100.00%
Epoch [7/10] Train Loss: 0.2746 Train Acc: 100.00% Val Acc: 100.00%
Epoch [8/10] Train Loss: 0.3792 Train Acc: 99.52% Val Acc: 100.00%
Epoch [9/10] Train Loss: 0.8952 Train Acc: 99.76% Val Acc: 100.00%
Epoch [10/10] Train Loss: 0.1919 Train Acc: 100.00% Val Acc: 100.00%
🎯 Training complete! Model saved as: imgtype_resnet18.pth
